# Making a cubic camembert

The moonshot: I have designed a camembert contact map. Let's see if it actually works!

In [1]:
import numpy as np
from json_dump import *
import contact_utils as cu
import config as cfg
from pathlib import Path
from geometry.cubic import CubicGeometry
from plotting.plot_cubic import plot_cubes_from_simulation_results

Found: /Applications/Blender.app/Contents/MacOS/Blender


In [2]:
### Key parameter: how we will name this run
run_name = "03_hairy_camembert"

### Define model parameters

model_params = {}

# ---------- LATTICE OPTIONS ----------

# Options:
# "chain", "square", "triangular", "cubic", "bcc", "fcc"
model_params["lattice_name"] = "cubic"

# Lattice dimensions
model_params["lx"] = 18
model_params["ly"] = 18  # Has to be 1 for chain
model_params["lz"] = 18  # Has to be 1 for square & triangular

In [3]:
# ---------- MODEL PARAMETERS ----------

# Number of particle types
model_params["n_types"] = 1

# Number of particles of each type
model_params["n_particles"] = [800]

# Couplings is its own beast. Should be gotten with the appropriate helper
# function.

## Important note: for 3D particles, do not use the get_matrix and set_contact utils!

A lot of face pairs are equivalent under rotation for cubes (e.g. (0, 0), (1, 3), (2, 2), (3, 1)).
When using a full contact matrix and feeding it to the wrapper using `set_single_species_contacts`, if all the equivalent pairs don't
have the same matrix coefficient, some of them will be overwritten by other values, and the contacts will be wrong.

Better to do things the way they are done below

In [4]:
crystal_energy = -10
# crystal_energy = 0.0
# defect_energy = -100
# defect_energy = crystal_energy * 2
defect_energy = 0
repel_energy = 10000
# crystal_energy = repel_energy

get_opposite_face = CubicGeometry().get_opposite_face
# First, create a cmap_wrapper object with the appropriate number of particle types
cmap_wrapper = cu.ContactMapWrapper.cubic(
    n_types = model_params["n_types"], init_energy = repel_energy)

# Setting crystalline interactions
for i in range(3):
    face = 4 * i
    opposite_face = get_opposite_face(face)
    cmap_wrapper[face, opposite_face] = crystal_energy

# Setting camembert interactions
camembert_contacts =  [
    (4, 12),
    (4, 13),
    (4, 14),
    (4, 15),
    (8, 12),
    (8, 13),
    (8, 14),
    (8, 15),
    (12, 20),
    (12, 21),
    (12, 22),
    (12, 23),
    (16, 12),
    (16, 13),
    (16, 14),
    (16, 15),
    (20, 12),
    (20, 13),
    (20, 14),
    (20, 15),
    ]
for contact in camembert_contacts:
    cmap_wrapper[contact] = defect_energy

# And process the couplings into flattened form
model_params["couplings"] = cmap_wrapper.get_formatted_couplings()

In [5]:
# Initialization option
model_params["initialize_option"] = "random"

# If "initialize_option" is set to "from_file", we must specify the location of
# the input file
# model_params["state_input"] = str(cfg.structures_path/"final_structure.dat")

# Options for average collection

model_params["state_av_option"] = True
model_params["e_av_option"] = True
model_params["e_record_option"] = False

if model_params["state_av_option"]:
    state_path = Path("./data/"+run_name+"/average_state")
    state_path.mkdir(parents = True, exist_ok = True)
    model_params["state_av_output"] =  str(state_path.resolve()) + "/"

if model_params["e_av_option"]:
    energy_path = Path("./data/"+run_name+"/average_energy")
    energy_path.mkdir(parents = True, exist_ok = True)
    model_params["e_av_output"] = str(energy_path.resolve())  + "/"

In [6]:
# Pick the probabilities of different moves. Has to sum to 1.
# Options:
""""
    "swap_empty_full",
    "swap_full_full",
    "rotate",
    "mutate",
    "rotate_and_swap_w_empty"
"""

moves_dict = {}
moves_dict["swap_empty_full"] = 1/3
moves_dict["rotate"] = 1/3
moves_dict["rotate_and_swap_w_empty"] = 1/3

model_params["move_probas"] = moves_dict

make_json_file(model_params, cfg.input_path/"model_params.json")

In [7]:
### Define mc parameters

mc_params = {}

# Number of MC steps used for equilibration
mc_params["mcs_eq"] = model_params["n_particles"][0] * 48

# Number of MC steps used for averaging
mc_params["mcs_av"] = 10

# Type of cooling schedule
# if exponential chosen: specify log10(T) as initial and final temperatures
mc_params["cooling_schedule"] = "exponential"

# Initial annealing temperature
mc_params["Ti"] = 2

# Final annealing temperature
mc_params["Tf"] = 0

# Number of annealing steps
mc_params["Nt"] = 40

# Option to collect state checkpoints at the end of each temperature cycle
mc_params["checkpoint_option"] = True

# If checkpoint is True, we need to provide the output address for the 
# checkpoint files
structures_path = Path("./data/"+run_name+"/structures")
structures_path.mkdir(parents = True, exist_ok = True)

if mc_params["checkpoint_option"]:
    mc_params["checkpoint_address"] = str(structures_path.resolve())+"/"

# Output location of the final state configuration (must end with "/")
mc_params["final_structure_address"] = str(structures_path.resolve())+"/"

make_json_file(mc_params, cfg.input_path/"mc_params.json")

In [ ]:
# Run the simulation from inside the notebook.
# Note that the program only writes to the cell output after it is done running.
# If you want to follow the excecution in real time, run the program from the command line!

# Parameter in run_sim is a boolean flag, which will overwrite the simulation results if set to True

cfg.run_simulation(overwrite = True)

At least one of the output folders is already populated!
overwrite flag set to True: running anyway.


In [40]:
# Plotting module creates a blender file which you have to open yourself
from plotting.plot_cubic import deleteAllObjects

blend_file_path = (f"/3dFigures/03_spiky_camembert_full_contacts_crystal_{crystal_energy:.02f}"
                   f"_rep_{repel_energy:.02f}_defect_{defect_energy:.02f}.blend")
print(blend_file_path)
plot_cubes_from_simulation_results(struct_folder = str(structures_path.resolve()), fig_file = blend_file_path)

/3dFigures/03_spiky_camembert_full_contacts_crystal_-10.00_rep_10000.00_defect_0.00.blend
800
f 'one_cube_numbered.obj' took 1.79 ms
OBJ import of 'one_cube_numbered.obj' took 1.48 ms
OBJ import of 'one_cube_numbered.obj' took 1.28 ms
OBJ import of 'one_cube_numbered.obj' took 1.41 ms
OBJ import of 'one_cube_numbered.obj' took 1.94 ms
OBJ import of 'one_cube_numbered.obj' took 1.01 ms
OBJ import of 'one_cube_numbered.obj' took 2.63 ms
OBJ import of 'one_cube_numbered.obj' took 1.10 ms
OBJ import of 'one_cube_numbered.obj' took 1.37 ms
OBJ import of 'one_cube_numbered.obj' took 1.58 ms
OBJ import of 'one_cube_numbered.obj' took 1.63 ms
OBJ import of 'one_cube_numbered.obj' took 1.48 ms
OBJ import of 'one_cube_numbered.obj' took 0.75 ms
OBJ import of 'one_cube_numbered.obj' took 1.39 ms
OBJ import of 'one_cube_numbered.obj' took 1.74 ms
OBJ import of 'one_cube_numbered.obj' took 1.38 ms
OBJ import of 'one_cube_numbered.obj' took 1.82 ms
OBJ import of 'one_cube_numbered.obj' took 1.17 ms


Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_number

umbered.obj' took 0.58 ms
OBJ import of 'one_cube_numbered.obj' took 0.29 ms
OBJ import of 'one_cube_numbered.obj' took 0.46 ms
OBJ import of 'one_cube_numbered.obj' took 0.47 ms
OBJ import of 'one_cube_numbered.obj' took 0.23 ms
OBJ import of 'one_cube_numbered.obj' took 0.52 ms
OBJ import of 'one_cube_numbered.obj' took 0.92 ms
OBJ import of 'one_cube_numbered.obj' took 0.28 ms
OBJ import of 'one_cube_numbered.obj' took 0.59 ms
OBJ import of 'one_cube_numbered.obj' took 0.76 ms
OBJ import of 'one_cube_numbered.obj' took 0.47 ms
OBJ import of 'one_cube_numbered.obj' took 0.52 ms
OBJ import of 'one_cube_numbered.obj' took 0.38 ms
OBJ import of 'one_cube_numbered.obj' took 0.50 ms
OBJ import of 'one_cube_numbered.obj' took 0.36 ms
OBJ import of 'one_cube_numbered.obj' took 0.33 ms
OBJ import of 'one_cube_numbered.obj' took 0.51 ms
OBJ import of 'one_cube_numbered.obj' took 0.67 ms
OBJ import of 'one_cube_numbered.obj' took 0.66 ms
OBJ import of 'one_cube_numbered.obj' took 0.28 ms
OBJ i

Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_number

 took 1.52 ms
OBJ import of 'one_cube_numbered.obj' took 1.27 ms
OBJ import of 'one_cube_numbered.obj' took 1.03 ms
OBJ import of 'one_cube_numbered.obj' took 1.19 ms
OBJ import of 'one_cube_numbered.obj' took 2.52 ms
OBJ import of 'one_cube_numbered.obj' took 3.31 ms
OBJ import of 'one_cube_numbered.obj' took 0.91 ms
OBJ import of 'one_cube_numbered.obj' took 0.91 ms
OBJ import of 'one_cube_numbered.obj' took 0.90 ms
OBJ import of 'one_cube_numbered.obj' took 1.29 ms
OBJ import of 'one_cube_numbered.obj' took 1.22 ms
OBJ import of 'one_cube_numbered.obj' took 0.93 ms
OBJ import of 'one_cube_numbered.obj' took 0.97 ms
OBJ import of 'one_cube_numbered.obj' took 1.01 ms
OBJ import of 'one_cube_numbered.obj' took 0.94 ms
OBJ import of 'one_cube_numbered.obj' took 0.95 ms
OBJ import of 'one_cube_numbered.obj' took 1.28 ms
OBJ import of 'one_cube_numbered.obj' took 1.20 ms
OBJ import of 'one_cube_numbered.obj' took 0.89 ms
OBJ import of 'one_cube_numbered.obj' took 1.16 ms
OBJ import of 'on

Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_number

Error: Cannot open file /3dFigures/03_spiky_camembert_full_contacts_crystal_-10.00_rep_10000.00_defect_0.00.blend@ for writing: No such file or directory


Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/camemberts3D/frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'


RuntimeError: Error: Cannot open file /3dFigures/03_spiky_camembert_full_contacts_crystal_-10.00_rep_10000.00_defect_0.00.blend@ for writing: No such file or directory


In [23]:
# Determining the aggregate sizes
# Using the linked-list method, because it's fun

sites_orientations = cfg.load_structure(struct_folder = structures_path)
n_sites = sites_orientations.shape[1]
full_sites = cfg.get_full_sites(sites_orientations)
n_particles = len(full_sites)

clusters = np.arange(len(full_sites))

# Link list method to build clusters
head = np.zeros(n_sites, dtype=int) - 1
lscl = np.zeros(n_particles, dtype=int) - 1

for i in range(n_particles):
    site = full_sites[i]
    lscl[i] = head[site]
    head[site] = i

In [26]:
class testClass:
    def __init__(self):
        self.x = 0

In [36]:
from geometry.cubic import CubicGeometry 
from geometry.geometry import (
lattice_coords_to_lattice_site_3d, 
lattice_site_to_lattice_coords_3d
)

lx = model_params["lx"]
ly = model_params["ly"]
lz = model_params["lz"]

cg = CubicGeometry()
for site in range(n_sites):
    site_coords = lattice_site_to_lattice_coords_3d(site, lx, ly)
    for bond in cg.bond_to_bond_index.keys():
        neighbour_coords = site_coords + bond


In [21]:
lscl

array([  -1,    0,    1, ..., 6628, 6629, 6630])

In [22]:
head

array([5831, 6631,   -1, ...,   -1,   -1,   -1])

In [24]:
head[0]

5831

In [25]:
lscl[head[0]]

5830

In [14]:
head

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1])